# Немного анализа данных - или как посчитать чужие деньги на PowerShell

Вы думали PowerShell — это только чтобы админить серверы? А вот и нет! Сегодня мы с вами займёмся благородным делом: **будем считать чужие деньги**. Да не где-нибудь, а по официальным данным американского регулятора SEC. И всё это — не отходя от консоли, с лёгким привкусом [Vega](https://vega.github.io) и аналитической дерзости.

> Форма 4 (Form 4) — это обязательная отчетность, которую заполняют директора, должностные лица (officers) и крупные акционеры (владельцы более 10% акций) публичной компании при совершении сделки с акциями этой компании.
>
> 📄 Основные сведения о Form 4:
> - **Кто подает**: инсайдеры компании (директора, топ-менеджеры, крупные акционеры).
> - **Когда подается**: в течение 2 рабочих дней после сделки с ценными бумагами.
> - **Что указывается**:
>     - имя инсайдера и его роль в компании;
>     - дата сделки;
>     - тип ценной бумаги (например, обыкновенные акции);
>     - количество акций, купленных или проданных;
>     - цена сделки;
>     - тип транзакции (покупка, продажа, передача, подарок и т.д.).

Для этого нам понадобятся две функции:

- `Get-RecentSecForm4XmlUrls` — как настоящий следователь, она пробирается в архив SEC и вытаскивает ссылки на XML-файлы форм.
- `Convert-Form4XmlToRecord` — читает XML и превращает его в внятный PowerShell-объект. Потому что глазами это читать — себе дороже.


In [1]:
<#
.SYNOPSIS
Retrieves a list of XML URLs for recent Form 4 insider filings from the SEC for a specified CIK.

.DESCRIPTION
This function queries the SEC EDGAR submissions API for a given company identified by its CIK (Central Index Key),
and returns a list of links to XML versions of Form 4 insider trading reports filed within the past N days.

.PARAMETER CIK
The Central Index Key (CIK) of the company. Defaults to Microsoft Corporation (0000789019).

.PARAMETER DaysBack
Number of days in the past to include filings. Defaults to 100 days.

.OUTPUTS
[PSCustomObject] with the following fields:
- FilingDate
- ReportDate
- XmlUrl

.EXAMPLE
Get-RecentSecForm4XmlUrls -CIK "0000320193" -DaysBack 30
Retrieves recent Form 4 XML links for Apple Inc. over the past 30 days.

.EXAMPLE
Get-RecentSecForm4XmlUrls
Returns recent Form 4 filings for Microsoft Corporation from the past 100 days.

.NOTES
A custom User-Agent header is required to access the SEC data endpoints.
#>

function Get-RecentSecForm4XmlUrls {
    param (
        [string]$CIK = "0000789019",
        [int]$DaysBack = 100
    )

    $headers = @{
        "User-Agent" = "PowerShellScript/1.0 (eosfor@gmail.com)"
        "Accept-Encoding" = "gzip, deflate"
    }

    $url = "https://data.sec.gov/submissions/CIK$CIK.json"
    $data = Invoke-RestMethod -Uri $url -Headers $headers

    $cikTrimmed = $CIK.TrimStart("0")
    $cutoffDate = (Get-Date).AddDays(-$DaysBack)

    $results = @()

    for ($i = 0; $i -lt $data.filings.recent.form.Length; $i++) {
        $formType = $data.filings.recent.form[$i]
        if ($formType -ne "4") { continue }

        $filingDate = Get-Date $data.filings.recent.filingDate[$i]
        if ($filingDate -lt $cutoffDate) { continue }

        $accessionNumber = $data.filings.recent.accessionNumber[$i]
        $primaryDoc = $data.filings.recent.primaryDocument[$i]
        $reportDate = $data.filings.recent.reportDate[$i]

        $folder = $accessionNumber -replace "-", ""
        $xmlFileName = [System.IO.Path]::GetFileNameWithoutExtension($primaryDoc) + ".xml"
        $xmlUrl = "https://www.sec.gov/Archives/edgar/data/$cikTrimmed/$folder/$xmlFileName"

        $results += [PSCustomObject]@{
            FilingDate = $filingDate.ToString("yyyy-MM-dd")
            ReportDate = $reportDate
            XmlUrl     = $xmlUrl
        }
    }

    return $results
}

In [2]:
<#
.SYNOPSIS
Converts a Form 4 XML document into a structured PowerShell object representing insider transactions.

.DESCRIPTION
This function takes an object with an XmlUrl (typically output from Get-RecentSecForm4XmlUrls), downloads the Form 4 XML,
and extracts detailed information about the issuer, insider, role, transaction type, number of shares, price,
ownership nature, and any associated footnotes.

.PARAMETER InputObject
An object containing XmlUrl, FilingDate, and ReportDate fields. Usually piped from Get-RecentSecForm4XmlUrls.

.OUTPUTS
[PSCustomObject] with the following fields:
- FilingDate
- ReportDate
- Issuer
- InsiderName
- InsiderRole
- SecurityTitle
- TransactionDate
- TransactionCode
- SharesTransacted
- PricePerShare
- SharesOwnedAfterTxn
- OwnershipType
- IndirectOwnershipNature
- Footnote
- XmlUrl

.EXAMPLE
Get-RecentSecForm4XmlUrls -CIK "0000789019" | Convert-Form4XmlToRecord
Returns parsed insider transactions for Microsoft Corporation.

.NOTES
Only non-derivative transactions are processed. If the XML cannot be downloaded, a warning is displayed.
#>

function Convert-Form4XmlToRecord {
    [CmdletBinding()]
    param (
        [Parameter(ValueFromPipeline = $true)]
        [pscustomobject]$InputObject
    )

    process {
        $headers = @{
            "User-Agent" = "PowerShellScript/1.0 (eosfor@gmail.com)"
        }

        try {
            [xml]$doc = Invoke-WebRequest -Uri $InputObject.XmlUrl -Headers $headers -UseBasicParsing
        }
        catch {
            Write-Warning "Download failed: $($InputObject.XmlUrl)"
            return
        }

        $issuer = $doc.ownershipDocument.issuer.issuerName
        $owner = $doc.ownershipDocument.reportingOwner.reportingOwnerId.rptOwnerName
        $ownerRelationship = $doc.ownershipDocument.reportingOwner.reportingOwnerRelationship
        $relationship = ($ownerRelationship | Get-Member -MemberType Properties | Where-Object { $ownerRelationship.$($_.Name) -eq "1" }).Name

        # Собираем footnotes в хештаблицу
        $footnotes = @{}

        if ($doc.ownershipDocument.footnotes -and $doc.ownershipDocument.footnotes.footnote) {
            $rawFootnotes = $doc.ownershipDocument.footnotes.footnote

            # Убедимся, что это массив
            if ($rawFootnotes -is [System.Array]) {
                foreach ($f in $rawFootnotes) {
                    $footnotes[$f.id] = $f.'#text' ?? $f.InnerText
                }
            }
            else {
                $footnotes[$rawFootnotes.id] = $rawFootnotes.'#text' ?? $rawFootnotes.InnerText
            }
        }

        $transactions = $doc.ownershipDocument.nonDerivativeTable.nonDerivativeTransaction
        foreach ($txn in $transactions) {
            # если есть одна или несколько сносок — собираем их все
            $note = $null
            if ($txn.footnoteId) {
                $ids = if ($txn.footnoteId -is [System.Array]) {
                    $txn.footnoteId | ForEach-Object { $_.id }
                } else {
                    @($txn.footnoteId.id)
                }

                $note = ($ids | ForEach-Object { $footnotes[$_] }) -join "; "
            }

            [PSCustomObject]@{
                FilingDate              = $InputObject.FilingDate
                ReportDate              = $InputObject.ReportDate
                Issuer                  = $issuer
                InsiderName             = $owner
                InsiderRole             = $relationship
                SecurityTitle           = $txn.securityTitle.value
                TransactionDate         = $txn.transactionDate.value
                TransactionCode         = $txn.transactionCoding.transactionCode
                SharesTransacted        = $txn.transactionAmounts.transactionShares.value
                PricePerShare           = $txn.transactionAmounts.transactionPricePerShare.value
                SharesOwnedAfterTxn     = $txn.postTransactionAmounts.sharesOwnedFollowingTransaction.value
                OwnershipType           = $txn.ownershipNature.directOrIndirectOwnership.value
                IndirectOwnershipNature = $txn.ownershipNature.natureOfOwnership.value
                Footnote                = $note
                XmlUrl                  = $InputObject.XmlUrl
            }
        }
    }
}

📥 Ну что, запускаем наш скрипт наблюдения и складываем всё в переменную `$allData`. Это как «навести справки», но без нарушения закона. Здесь вы можете задать свой CIK

In [3]:
$allData = 
Get-RecentSecForm4XmlUrls -CIK "0000789019" -DaysBack 107 |
    Convert-Form4XmlToRecord

🧹 Теперь немного наведём порядок — отберём только те сделки, где **деньги действительно ходили**. Если акций в строке 0 — мимо. Нам нужны настоящие миллионы (ну или хотя бы парочка лотов).

- $allData — это массив объектов, содержащих данные о транзакциях
- Select-Object выбирает только нужные поля: TransactionDate, SharesTransacted, TransactionCode.
- Where-Object фильтрует:
    - транзакции, у которых TransactionCode входит в список допустимых кодов
    - количество акций (SharesTransacted) должно быть больше 0.

In [4]:
$data = $allData |
    Select-Object TransactionDate, SharesTransacted, TransactionCode |
    Where-Object { $_.TransactionCode -in @("S", "P", "F", "A", "M", "G") -and $_.SharesTransacted -gt 0 }

$data = $data | ForEach-Object {
    $action = switch ($_.TransactionCode) {
        "S" { "Sell"; break }
        "F" { "Sell"; break }
        "G" { "Sell"; break }
        "A" { "Buy"; break }
        "P" { "Buy"; break }
        "M" { "Buy"; break }
        default { "Other" }
    }

    $_ | Add-Member -NotePropertyName Action -NotePropertyValue $action -Force -PassThru
}

📊 Следующий этап — **агрегируем, кто сколько успел прикупить или продать**. Группируем данные по имени инсайдера и типу сделки. Если цена есть — считаем сумму. Если нет — ну, значит, будет 'неизвестно', как в старой доброй бухгалтерии.

In [5]:
$data2 = $allData |
    # Filter only transactions with non-zero number of shares
    Where-Object { $_.SharesTransacted -gt 0 } |

    # Group by InsiderName and TransactionCode (e.g., "John Smith|S")
    Group-Object { "$($_.InsiderName)|$($_.TransactionCode)" } |

    ForEach-Object {
        $parts = $_.Name -split '\|'     # Split group name into [InsiderName, TransactionCode]
        $group = $_.Group                # Access the actual group of transactions

        # Filter only those deals with valid numeric and positive price per share
        $validDeals = $group | Where-Object {
            [double]::TryParse($_.PricePerShare, [ref]$null) -and [double]$_.PricePerShare -gt 0
        }

        # Sum all shares transacted in the group
        $sharesSum = ($group | Measure-Object -Property SharesTransacted -Sum).Sum

        # Calculate total value by summing (Shares × Price) across valid deals
        $totalValue = ($validDeals | ForEach-Object {
            [double]$_.SharesTransacted * [double]$_.PricePerShare
        }) | Measure-Object -Sum | Select-Object -ExpandProperty Sum

        # Display value only if valid (not null or NaN)
        $valueDisplay = if ($totalValue -and $totalValue -gt 0 -and -not [double]::IsNaN($totalValue)) {
            [math]::Round($totalValue, 2)
        } else {
            "unknown"
        }

        # Return summary object for each (InsiderName, TransactionCode) group
        [PSCustomObject]@{
            Insider            = $parts[0]
            TransactionCode    = $parts[1]
            Count              = $_.Count
            TotalShares        = [math]::Round($sharesSum, 2)
            TotalValue         = if ($valueDisplay -is [string]) { $null } else { $valueDisplay }
            TotalValueDisplay  = "$valueDisplay"
        }
    }

📋 А тут мы просто представляем каждую сделку как отдельную строку: дата, кто, что, сколько. Если цена указана — хорошо. Не указана — ну, вы поняли: 'неизвестно'. Главное — не упустить никого из поля зрения 🕵️‍♂️

In [6]:
$data3 = $allData |
    # Filter transactions with shares > 0 and valid date format (YYYY-MM-DD)
    Where-Object {
        $_.SharesTransacted -gt 0 -and
        $_.TransactionDate -match '^\d{4}-\d{2}-\d{2}$'
    } |

    # Process each valid transaction
    ForEach-Object {
        $value = $null              # Holds calculated total value (shares * price)
        $display = "unknown"     # Default display if value is unknown

        # Calculate total value if price is valid and > 0
        if ([double]::TryParse($_.PricePerShare, [ref]$null) -and [double]$_.PricePerShare -gt 0) {
            $value = [math]::Round([double]$_.SharesTransacted * [double]$_.PricePerShare, 2)
            $display = "$value"     # Use calculated value for display
        }

        # Return a simplified transaction record
        [PSCustomObject]@{
            Insider            = $_.InsiderName
            TransactionDate    = $_.TransactionDate
            TransactionCode    = $_.TransactionCode
            SharesTransacted   = [int]$_.SharesTransacted
            TotalValue         = $value
            TotalValueDisplay  = $display
        }
    }

💾 Финальный аккорд — выгружаем наши драгоценные данные в CSV, чтобы потом рисовать красивые графики и всем показывать, что PowerShell — это не только `Get-Process`.

In [7]:
$allData | Export-Csv allData.csv -NoTypeInformation
$data | Export-Csv -Path "form4-trades.csv" -NoTypeInformation
$data2 | Export-Csv -Path "insider-heatmap.csv" -NoTypeInformation
$data3 | Export-Csv -NoTypeInformation -Path "insider-scatter.csv"

🔧 Чуть не забыли! Чтобы всё это работало, пришлось внести вклад в сам `dotnet/interactive`. Почему? Потому что параметр `CustomMimeType` в `Out-Display` раньше был... ну, типа был, но как бы не работал. Теперь работает, можно подавать JSON-спеку прямо из ячейки и видеть графики в цвете. Можете сказать спасибо автору [PR #3671](https://github.com/dotnet/interactive/pull/3671), то-есть мне 😉

📈 **Scatter Plot** — наш первый визуальный допрос:

- X — дата сделки  
- Y — количество акций  
- Цвет — зелёный (купили) или красный (продали)  
- Подсказки — кто, когда, сколько и какой буквой это всё обозначено  

Простой способ понять, **кто что знал и когда решил продать** 💸

Выглядит это примерно вот так

In [8]:
@"
{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "Insider Trading Scatter Plot",
  "data": {
    "url": "form4-trades.csv",
    "format": {"type": "csv"}
  },
  "mark": "point",
  "encoding": {
    "x": {
      "field": "TransactionDate",
      "type": "temporal",
      "title": "Дата"
    },
    "y": {
      "field": "SharesTransacted",
      "type": "quantitative",
      "title": "Кол-во акций"
    },
    "color": {
      "field": "Action",
      "type": "nominal",
      "scale": {
        "domain": ["Buy", "Sell"],
        "range": ["green", "red"]
      },
      "title": "Тип сделки"
    },
    "tooltip": [
      {"field": "TransactionDate", "type": "temporal"},
      {"field": "SharesTransacted", "type": "quantitative"},
      {"field": "TransactionCode", "type": "nominal"}
    ]
  }
}
"@ | Out-Display -MimeType "application/vnd.vegalite.v5+json"

### 🔍 Расшифровка `TransactionCode`

| Код | Что это            | Как это понимать                                                        |
|-----|--------------------|--------------------------------------------------------------------------|
| A   | Award              | Начислили акции, типа премия. Как авансом, только в виде доли компании. |
| S   | Sale               | Продал. Часто — массово. Иногда — перед падением котировок. Хмм...      |
| F   | Tax                | Часть акций ушла на налоги. Ну, хотя бы не себе.                        |
| M   | Option Exercise    | Реализовал опцион. Купил подешевле — продал подороже. Классика.         |
| G   | Gift               | Подарил. Родным. Или в траст. Или на фонд. Мы не осуждаем.              |
| P   | Purchase           | Купил. На свои. Молодец.                                                |
| I   | Discretionary      | Автоматическая сделка, типа по плану. Можно верить, можно нет.          |
| C   | Conversion         | Преобразование чего-то мутного (деривативов) в простые акции. Всё по регламенту. |

🔥 **Heatmap** — где жарко, там и инсайдер:

- X — тип сделки  
- Y — кто  
- Цвет — зелёный, если сумма есть, серый — если неизвестно  
- Tooltip — сколько сделок, акций и на какую сумму  

Любой может ошибиться, но только не тепловая карта 💼

In [10]:
@"
{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "Heatmap of Insider Transaction Totals",
  "data": {
    "url": "insider-heatmap.csv",
    "format": {"type": "csv"}
  },
  "mark": "rect",
  "encoding": {
    "x": {
      "field": "TransactionCode",
      "type": "nominal",
      "title": "Тип сделки"
    },
    "y": {
      "field": "Insider",
      "type": "nominal",
      "title": "Инсайдер",
      "sort": "-x"
    },
    "color": {
      "condition": {
        "test": "isValid(datum.TotalValue) && datum.TotalValue != ''",
        "field": "TotalValue",
        "type": "quantitative",
        "scale": { "scheme": "greens" }
      },
      "value": "#eeeeee"
    },
    "tooltip": [
      { "field": "Insider", "type": "nominal", "title": "Инсайдер" },
      { "field": "TransactionCode", "type": "nominal", "title": "Код сделки" },
      { "field": "Count", "type": "quantitative", "title": "Кол-во сделок" },
      { "field": "TotalShares", "type": "quantitative", "title": "Всего акций" },
      { "field": "TotalValueDisplay", "type": "nominal", "title": "Сумма ($)" }
    ]
  },
  "config": {
    "axis": {
      "labelFontSize": 10,
      "titleFontSize": 12
    },
    "view": {
      "stroke": "transparent"
    }
  }
}
"@  | Out-Display -MimeType "application/vnd.vegalite.v5+json"

🔵 **Bubble Chart** — пузырьковая диаграмма, где каждая точка — сделка, а размер — количество акций. Чем жирнее — тем вкуснее:

- X — дата  
- Y — кто  
- Размер — сколько акций  
- Цвет — тип сделки  
- Tooltip — вся подноготная  

Отлично видно, кто от жадности лопнул первым 😄

In [11]:
@"
{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
  "description": "Bubble Chart: Insider vs Date vs Transaction Type",
  "width": 800,
  "data": {
    "url": "insider-scatter.csv",
    "format": { "type": "csv" }
  },
  "transform": [
    {
      "lookup": "TransactionCode",
      "from": {
        "data": {
        "values": [
          { "code": "A", "label": "Award" },
          { "code": "S", "label": "Sale" },
          { "code": "F", "label": "Tax" },
          { "code": "M", "label": "Exercise" },
          { "code": "G", "label": "Gift" },
          { "code": "P", "label": "Purchase" },
          { "code": "I", "label": "Discretionary" },
          { "code": "C", "label": "Conversion" }
        ]
        },
        "key": "code",
        "fields": ["label"]
      },
      "default": "Unknown"
    }
  ],
  "mark": {
    "type": "circle",
    "opacity": 0.7
  },
  "encoding": {
    "x": {
      "field": "TransactionDate",
      "type": "temporal",
      "title": "Дата сделки"
    },
    "y": {
      "field": "Insider",
      "type": "nominal",
      "title": "Инсайдер",
      "sort": "-x"
    },
    "color": {
      "field": "label",
      "type": "nominal",
      "title": "Тип сделки"
    },
    "size": {
      "field": "SharesTransacted",
      "type": "quantitative",
      "title": "Кол-во акций"
    },
    "tooltip": [
      { "field": "Insider", "type": "nominal", "title": "Инсайдер" },
      { "field": "TransactionDate", "type": "temporal", "title": "Дата" },
      { "field": "TransactionCode", "type": "nominal", "title": "Код сделки" },
      { "field": "SharesTransacted", "type": "quantitative", "title": "Кол-во акций" },
      { "field": "TotalValueDisplay", "type": "nominal", "title": "Сумма ($)" }
    ]
  },
  "config": {
    "axis": {
      "labelFontSize": 10,
      "titleFontSize": 12
    },
    "legend": {
      "labelFontSize": 10,
      "titleFontSize": 12
    },
    "view": {
      "stroke": "transparent"
    }
  }
}
"@ | Out-Display -MimeType "application/vnd.vegalite.v5+json"